In [2]:
##Some code refered to Basil
import  pandas  as  pd
import  numpy  as  np

path='D:/4th Semester/Data Mining2/DMC/Small/'
data=pd.read_csv(path+'data_v0.1_sales.csv')
# filter small products
daten=data[(data.cluster=='small')]
#seperate data in (october - december) and january
data_target=daten.iloc[:,93:124]
data_train =daten.iloc[:,0:93]
data_train.shape

(9917, 93)

In [3]:
#read test data
test0=pd.read_csv(path+'test_0.csv')
test1=pd.read_csv(path+'test_1.csv')
test2=pd.read_csv(path+'test_2.csv')
test3=pd.read_csv(path+'test_3.csv')
test4=pd.read_csv(path+'test_4.csv')

#change type from float to int as preparation for key generation
test0['pid']=test0['pid'].astype(int)
test1['pid']=test1['pid'].astype(int)
test2['pid']=test2['pid'].astype(int)
test3['pid']=test3['pid'].astype(int)
test4['pid']=test4['pid'].astype(int)

#create key function
def keycr(train):
    keys3=[]
    i=0
    while i<len(train):
        key=str(train.iloc[i,1])+str(train.iloc[i,2])
        keys3.append(key)
        i=i+1
    return keys3


#creating the keys for test data#creating 
test0['key']=keycr(test0)
test1['key']=keycr(test1)
test2['key']=keycr(test2)
test3['key']=keycr(test3)
test4['key']=keycr(test4)


#compute avg sales per product#compute  
data_train['avg']=data_train.mean(axis=1)
data_train = data_train.reset_index(drop=True)

# function creates new test data set which includes only the small product
def seperate_test(test):
    i=0 
    arr=[]
    c=0
    test = test.reset_index(drop=True)
    while i<len(data_train):
        if data_train['key'][i] in test['key'].values:
            arr.append(data_train['key'][i])
            c=c+1
        i=i+1
    print('NUMBER OF KEYS OF SMALL PRODUCTS IN TEST')
    print(c)
    i=0 
    test00=test
    while i<len(test):
        if test['key'][i] not in arr:
            test00=test00[test00.key != test['key'][i]]
        i=i+1
    return test00


#applying the function#applying 
test0=seperate_test(test0)
test1=seperate_test(test1)
test2=seperate_test(test2)
test3=seperate_test(test3)
test4=seperate_test(test4)

NUMBER OF KEYS OF SMALL PRODUCTS IN TEST
5505
NUMBER OF KEYS OF SMALL PRODUCTS IN TEST
5505
NUMBER OF KEYS OF SMALL PRODUCTS IN TEST
5505
NUMBER OF KEYS OF SMALL PRODUCTS IN TEST
5505
NUMBER OF KEYS OF SMALL PRODUCTS IN TEST
5505


In [4]:
data_train['avg'].head()

0    0.010870
1    0.021739
2    0.032609
3    0.108696
4    0.010870
Name: avg, dtype: float64

In [5]:
#baseline 1 use the daily average of all and fb prophet 
daily_avg=data_train.mean()
daily_avg=np.array(daily_avg)
daily_avg=daily_avg[0:92]
len(daily_avg)

92

In [6]:
data_train.iloc[0:10 ,93]

0    0.010870
1    0.021739
2    0.032609
3    0.108696
4    0.010870
5    0.021739
6    0.000000
7    0.010870
8    0.010870
9    0.021739
Name: avg, dtype: float64

In [7]:
i=data_train.loc[data_train['key'] == '10001L']
i.iloc[0,92]

0.0

In [36]:
#compute the average of the same date of each month
#i is the dataframe with a single key
def month_daily(i):
    month_day=[]
    for d in range(0,30):
        n=(i.iloc[0,d+1]+i.iloc[0,d+32]+i.iloc[0,d+62])/3
        month_day.append(n)
    month_day.append((i.iloc[0,32]+i.iloc[0,92])/2)
    return month_day


def soldout_day(pred, stock):
    """
    Calculates first day that stock hits 0 in a certain month for an item
    :param pred: Array of predicted sales units for an item
    :param stock: Stock at beginning of month for an item
    :return: Day of month that stock reaches 0
    """
    soldout_day = len(pred)
    for day in range(len(pred)):
        stock -= pred[day]
        #print(stock)
        if stock <= 0:
            soldout_day = day+1
            break
    return soldout_day


i=data_train.loc[data_train['key'] == '10001L']
m=month_daily(i)
pred_day=soldout_day(m,test0['stock'].iloc[0])
real_day=int(test0['sold_out_date'][0].split('-')[2])
gap=abs(pred_day-real_day)
gap

24

In [178]:
total_gap=[]
n=0
for index in range(len(test0)):
    key=test0['key'].iloc[index]
    i=data_train.loc[data_train['key'] == key]
    m=month_daily(i)
    if m.count(0)>=22:
        pred_day=18
        n+=1
    else: 
        pred_day=soldout_day(m,test0['stock'].iloc[index])
    real_day=int(test0['sold_out_date'].iloc[index].split('-')[2])
    gap=abs(pred_day-real_day)
    total_gap.append(gap)
import math
print('best score for avg of the same day in each month')
math.sqrt(sum(total_gap))


best score for avg of the same day in each month


210.8838542895117

In [179]:
##Window Function
total_gap=[]
n=0
for index in range(len(test0)):
    key=test0['key'].iloc[index]
    i=data_train.loc[data_train['key'] == key]
    data=np.array(i)[0]
    count=list(data[1:93]).count(0)
    p=i.drop(['key','avg'], axis=1).rolling(window=31, min_periods=1, freq=None, center=False, win_type='bartlett', on=None, axis=1, closed=None)
    m=np.array(p.mean())[0,61:]
    if count>=82:
        pred_day=18
        n+=1
    else:
        pred_day=soldout_day(m,test0['stock'].iloc[index])
    real_day=int(test0['sold_out_date'].iloc[index].split('-')[2])
    gap=abs(pred_day-real_day)
    total_gap.append(gap)
print('number of items which contains too many zeros')
print(n)
print('best score of window approach')
math.sqrt(sum(total_gap))

number of items which contains too many zeros
5438
best score of window approach


210.9099333838973